In [1]:
from datasets import load_dataset, load_metric, Audio,concatenate_datasets


In [2]:
import pandas as pd
import random

In [3]:
# train = pd.read_csv("cv-corpus-15.0-2023-09-08/as/train.tsv",sep='\t')
# validation = pd.read_csv("cv-corpus-15.0-2023-09-08/as/validated.tsv",sep='\t')
# common_voice_test = pd.read_csv("cv-corpus-15.0-2023-09-08/as/test.tsv",sep='\t')

In [4]:
data_files = {"train": "train.tsv", "validation": "dev.tsv"}
common_voice_train = load_dataset("cv-corpus-15.0-2023-09-08/as", data_files=data_files)
common_voice_train = concatenate_datasets([common_voice_train['train'],common_voice_train['validation']])
common_voice_test = load_dataset("cv-corpus-15.0-2023-09-08/as", split="test")

In [5]:
clips = load_dataset("cv-corpus-15.0-2023-09-08/as/clips",split='train')

Resolving data files:   0%|          | 0/2071 [00:00<?, ?it/s]

In [6]:
clips = clips.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
common_voice_train[0]

{'client_id': '062a55bf8558d9613788c5fd5e88899ba243bdb6ec92de1a11d8be75dc144b9a96dcd79e863c3144fd8ce38e5d1e4ec3d6152186c0296fb2fbdd42c085064217',
 'path': 'common_voice_as_32840924.mp3',
 'sentence': 'মাত শুনি ৰূপহী ওলাই আহি ক\'লে- "তেওঁ ঘৰত নাই।"',
 'up_votes': 4,
 'down_votes': 0,
 'age': 'twenties',
 'gender': 'male',
 'accents': 'Gaonlia',
 'variant': None,
 'locale': 'as',
 'segment': None}

In [8]:
common_voice_train.shape

(1166, 11)

In [9]:
common_voice_train.features


{'client_id': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'up_votes': Value(dtype='int64', id=None),
 'down_votes': Value(dtype='int64', id=None),
 'age': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'accents': Value(dtype='string', id=None),
 'variant': Value(dtype='float64', id=None),
 'locale': Value(dtype='string', id=None),
 'segment': Value(dtype='float64', id=None)}

In [10]:
common_voice_train = common_voice_train.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [11]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(common_voice_train.remove_columns(["path","variant"]), num_examples=10)

,sentence
0,আকাশত দুই-এটা তৰাই মাথোন ভুমুকি মাৰিছে।
1,"তাৰে গাঁৱৰ এঘৰত, ভেটিৰ পৰা আঁতৰি নাযাওঁ বুলি"
2,বিবাহৰ পৰৱৰ্তী জীৱনত তুমি কেতিয়াও সুখী হ'ব নোৱাৰিবা।
3,কী-ব'ৰ্ড আৰু অন্য ইনপুট ডিভাইচ।
4,কাইলে' চবেই ড্ৰ'ইং বুক আৰু কালাৰ পেঞ্চিল লৈ আনিবা।
5,মই তেজপুৰলৈ গৈছিলোঁ।
6,কি কৰিব! আমি ছোৱালী নিদিওঁ বুলি কোৱা নাই।
7,আজিয়েই বিয়াৰ দিন আছিল।
8,"ক'ত আছে, তেনেহ'লে তোক মই ক্ষমা কৰিম।"
9,এইবোৰ বিষয়ে তেওঁ সকলোৰে মতামত ল'লে।


In [13]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\।]'

In [14]:
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, "", batch["sentence"]).lower()
    return batch

In [15]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [16]:
show_random_elements(common_voice_train.remove_columns(["path","variant"]))

,sentence
0,উলৰ কাপোৰ গুণাৰ কাপোৰ অলঙ্কাৰ আদিৰ নিচিনা
1,ইতিমধ্যে তেজীমলাৰ সখীয়েকৰ বিয়া আহিল পৰহিলৈ বিয়া
2,এই আঙঠিটো কলপাত এখনত থৈ ভালমতে সেৱাশুশ্ৰষা কৰিলে
3,এতিয়া সি তেজপুৰহে নালাগে কি জানি অসমতেই নাই
4,এনেকৈয়ে আৰু ডেৰ বছৰমান যোৱাত ভদ্ৰৰ মৃত্যু হ'ল
5,চোৰ আৰু ক'ত ওলাব
6,ৰাতি দুপৰ হৈছে
7,দৰাক দেখা পাই হালিৰ বিষম খং উঠিল
8,সিহঁতে জানিলেহে
9,তেওঁলোকৰ বেচি ভাগৰে থকা ঘৰৰ ভেটিটোৰ বাহিৰে আন মাটিবাৰী নাই


In [17]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [18]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/1166 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

In [19]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [20]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 "'": 1,
 'ঁ': 2,
 'ং': 3,
 'ঃ': 4,
 'অ': 5,
 'আ': 6,
 'ই': 7,
 'ঈ': 8,
 'উ': 9,
 'এ': 10,
 'ঐ': 11,
 'ও': 12,
 'ঔ': 13,
 'ক': 14,
 'খ': 15,
 'গ': 16,
 'ঘ': 17,
 'ঙ': 18,
 'চ': 19,
 'ছ': 20,
 'জ': 21,
 'ঝ': 22,
 'ঞ': 23,
 'ট': 24,
 'ঠ': 25,
 'ড': 26,
 'ঢ': 27,
 'ণ': 28,
 'ত': 29,
 'থ': 30,
 'দ': 31,
 'ধ': 32,
 'ন': 33,
 'প': 34,
 'ফ': 35,
 'ব': 36,
 'ভ': 37,
 'ম': 38,
 'য': 39,
 'র': 40,
 'ল': 41,
 'শ': 42,
 'ষ': 43,
 'স': 44,
 'হ': 45,
 '়': 46,
 'া': 47,
 'ি': 48,
 'ী': 49,
 'ু': 50,
 'ূ': 51,
 'ৃ': 52,
 'ে': 53,
 'ৈ': 54,
 'ো': 55,
 'ৌ': 56,
 '্': 57,
 'ৎ': 58,
 'ড়': 59,
 'ঢ়': 60,
 'য়': 61,
 'ৰ': 62,
 'ৱ': 63,
 '’': 64}

In [21]:
vocab_dict['|'] = vocab_dict[' ']
del vocab_dict[' ']

In [22]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

67

In [23]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [24]:
from transformers import Wav2Vec2CTCTokenizer

In [25]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
from transformers import Wav2Vec2FeatureExtractor


In [27]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [28]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [29]:
common_voice_train[0]['path']

'common_voice_as_32840924.mp3'

In [30]:
common_voice_train[0]

{'path': 'common_voice_as_32840924.mp3',
 'sentence': "মাত শুনি ৰূপহী ওলাই আহি ক'লে তেওঁ ঘৰত নাই",
 'variant': None}

In [31]:
# clips['audio']

In [32]:
def find_audio(dictionary,path):
    for d in dictionary:
        if path in d['path']:
            return d

In [33]:
find_audio(clips['audio'],common_voice_train[0]['path'])

{'path': '/home/neeraj/Documents/AsrInJupyter/cv-corpus-15.0-2023-09-08/as/clips/common_voice_as_32840924.mp3',
 'array': array([ 1.81898940e-12,  1.05501385e-10,  0.00000000e+00, ...,
         1.15675959e-07,  6.64782789e-08, -3.15510391e-08]),
 'sampling_rate': 16000}

In [34]:
rand_int = random.randint(0, len(common_voice_train)-1)
rand_int

303

In [35]:
random_train_data =  common_voice_train[rand_int]


In [36]:
audio_data = find_audio(clips['audio'],random_train_data['path'])

In [37]:
print(random_train_data)

{'path': 'common_voice_as_32850458.mp3', 'sentence': 'আজি বহাগৰ ঘাই দিন', 'variant': None}


In [38]:
print(audio_data)

{'path': '/home/neeraj/Documents/AsrInJupyter/cv-corpus-15.0-2023-09-08/as/clips/common_voice_as_32850458.mp3', 'array': array([-5.68434189e-14,  3.97903932e-13,  1.98951966e-13, ...,
       -1.79738890e-09, -4.00064835e-08, -9.00213166e-08]), 'sampling_rate': 16000}


In [39]:
print("Target text:", random_train_data["sentence"])
print("Input array shape:", audio_data['array'].shape)
print("Sampling rate:", audio_data['sampling_rate'])

Target text: আজি বহাগৰ ঘাই দিন
Input array shape: (69120,)
Sampling rate: 16000


In [40]:
%%time
rand_int = random.randint(0, len(common_voice_train)-1)
random_train_data =  common_voice_train[rand_int]
audio_data = find_audio(clips['audio'],random_train_data['path'])
print("Target text:", random_train_data["sentence"])
print("Input array shape:", audio_data['array'].shape)
print("Sampling rate:", audio_data['sampling_rate'])

Target text: দোকানৰ পৰা সুভদ্ৰাৰ ঘৰলৈ মাত্ৰ তিনচাৰি মিনিটৰ বাট
Input array shape: (144000,)
Sampling rate: 16000
CPU times: user 7.79 s, sys: 132 ms, total: 7.93 s
Wall time: 7.92 s


In [41]:
import IPython.display as ipd

In [42]:
ipd.Audio(data=audio_data["array"], autoplay=True, rate=16000)

In [43]:
import os
import librosa

In [44]:
def add_audio(x):
    audio_path = os.path.join("cv-corpus-15.0-2023-09-08/as/clips",x['path'])
    audio_data,sr = librosa.load(audio_path,sr=16000)
    return {'audio': {"array":audio_data,'sampling_rate':sr}}

In [45]:
common_voice_train = common_voice_train.map(lambda x : add_audio(x))


In [46]:
common_voice_test = common_voice_test.map(lambda x: add_audio(x))

In [47]:
common_voice_test.features

{'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'variant': Value(dtype='float64', id=None),
 'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'sampling_rate': Value(dtype='int64', id=None)}}

In [48]:
common_voice_train.features

{'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'variant': Value(dtype='float64', id=None),
 'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'sampling_rate': Value(dtype='int64', id=None)}}

In [49]:
common_voice_train[0]['audio']['array']

[1.8189894035458565e-12,
 1.0550138540565968e-10,
 0.0,
 -2.546585164964199e-11,
 4.3655745685100555e-11,
 1.4551915228366852e-11,
 7.275957614183426e-12,
 -5.4569682106375694e-11,
 -1.4551915228366852e-11,
 -1.1823431123048067e-10,
 -2.9467628337442875e-10,
 -4.001776687800884e-10,
 -3.4560798667371273e-10,
 -3.80168785341084e-10,
 -2.4010660126805305e-10,
 -1.4915713109076023e-10,
 1.2369127944111824e-10,
 -1.0186340659856796e-10,
 2.1827872842550278e-10,
 5.093170329928398e-10,
 5.529727786779404e-10,
 4.94765117764473e-10,
 1.2369127944111824e-09,
 1.82626536116004e-09,
 1.9226717995479703e-09,
 1.979060471057892e-09,
 1.7898855730891228e-09,
 6.83940015733242e-10,
 1.8917489796876907e-10,
 7.130438461899757e-10,
 6.039044819772243e-10,
 -3.2014213502407074e-10,
 1.3969838619232178e-09,
 2.5247572921216488e-09,
 1.4842953532934189e-09,
 6.184563972055912e-10,
 -2.5793269742280245e-09,
 -7.043126970529556e-09,
 -1.1408701539039612e-08,
 -1.1757947504520416e-08,
 -1.4733814168721437e

In [50]:
common_voice_train[0]['sentence']

"মাত শুনি ৰূপহী ওলাই আহি ক'লে তেওঁ ঘৰত নাই"

In [51]:
# ipd.Audio(data=common_voice_train[0]['audio'], autoplay=True, rate=16000)


In [52]:
# common_voice_train.save_to_disk('common_voice_train_with_audio.arrow')

In [53]:
# common_voice_train[0]['audio']

In [54]:
def prepare_dataset(batch):
    audio = batch['audio']
    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch    

In [55]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [56]:
common_voice_train.features

{'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'input_length': Value(dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [57]:
common_voice_train[0]['input_values']

[4.695964162237942e-05,
 4.696078030974604e-05,
 4.695961979450658e-05,
 4.6959339670138434e-05,
 4.6960100007709116e-05,
 4.695977986557409e-05,
 4.6959699830040336e-05,
 4.695901952800341e-05,
 4.695945972343907e-05,
 4.695832103607245e-05,
 4.6956385631347075e-05,
 4.695522511610761e-05,
 4.695582538261078e-05,
 4.6955447032814845e-05,
 4.695698225987144e-05,
 4.695798270404339e-05,
 4.696098039858043e-05,
 4.69585029350128e-05,
 4.696201722254045e-05,
 4.696521500591189e-05,
 4.6965691581135616e-05,
 4.6965054934844375e-05,
 4.697320400737226e-05,
 4.6979675971670076e-05,
 4.698073462350294e-05,
 4.6981353079900146e-05,
 4.6979275794001296e-05,
 4.6967132220743224e-05,
 4.696169708040543e-05,
 4.696745236287825e-05,
 4.696625182987191e-05,
 4.695610550697893e-05,
 4.6974961151136085e-05,
 4.698734846897423e-05,
 4.697592157754116e-05,
 4.696641190093942e-05,
 4.6931294491514564e-05,
 4.6882272727089e-05,
 4.683432780439034e-05,
 4.6830493374727666e-05,
 4.679781341110356e-05,
 4.68

In [58]:
common_voice_train[0]['input_length']

92160

In [59]:
common_voice_train[0]['labels']

[38,
 47,
 29,
 0,
 42,
 50,
 33,
 48,
 0,
 62,
 51,
 34,
 45,
 49,
 0,
 12,
 41,
 47,
 7,
 0,
 6,
 45,
 48,
 0,
 14,
 1,
 41,
 53,
 0,
 29,
 53,
 12,
 2,
 0,
 17,
 62,
 29,
 0,
 33,
 47,
 7]

In [60]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [61]:
@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [62]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [63]:
data_collator

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=67, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]}, clean_up_tokenization_spaces=True), padding=True)

In [64]:
wer_metric = load_metric("wer")

/tmp/ipykernel_6820/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [65]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [66]:
from transformers import Wav2Vec2ForCTC

In [67]:
# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-xls-r-300m", 
#     attention_dropout=0.0,
#     hidden_dropout=0.0,
#     feat_proj_dropout=0.0,
#     mask_time_prob=0.05,
#     layerdrop=0.0,
#     ctc_loss_reduction="mean", 
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )

In [132]:
# model.save_pretrained("my-wav2vec2-model")

In [69]:
model = Wav2Vec2ForCTC.from_pretrained('my-wav2vec2-model')

In [70]:
model.freeze_feature_extractor()

/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1914: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [71]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [72]:
from transformers import TrainingArguments

In [79]:
training_args = TrainingArguments(
  output_dir='xlsr-wav2vec2-for-assamese',
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=False,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

In [80]:
from transformers import Trainer

In [81]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [82]:
import numpy as np

In [83]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
400,0.554900,1.005279,0.681526
800,0.188000,1.049076,0.596048


/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/neeraj/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecat

TrainOutput(global_step=1080, training_loss=0.2953873740302192, metrics={'train_runtime': 69970.9786, 'train_samples_per_second': 0.5, 'train_steps_per_second': 0.015, 'total_flos': 6.091161900006724e+18, 'train_loss': 0.2953873740302192, 'epoch': 29.59})

In [84]:
trainer.save_model('Saved_trainer/')

In [115]:
trainer

In [116]:
from huggingface_hub import notebook_login


In [88]:
notebook_login()

In [89]:
repo = 'xlsr-wav2vec2-for-assamese'

In [106]:
input_dict = processor(common_voice_test[1]["input_values"], return_tensors="pt", padding=True,sampling_rate=16000)


In [107]:
logits = model(input_dict.input_values).logits


In [108]:
pred_ids = torch.argmax(logits, dim=-1)[0]

In [102]:

common_voice_test_transcription = load_dataset("cv-corpus-15.0-2023-09-08/as", split="test")

In [109]:
print(processor.decode(pred_ids))

মো ভালো লগা নাই বেয়াও লগা না


In [110]:
print(common_voice_test_transcription[1]['sentence'].lower())

মন ভালো লগা নাই, বেয়াও, নগা নাই।


In [121]:
def decoded_voice(encoded):
    input_dict = processor(encoded, return_tensors="pt", padding=True,sampling_rate=16000)
    logits = model(input_dict.input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    return processor.decode(pred_ids)

In [122]:
decoded_voice(common_voice_test[1]["input_values"])

'মোন ভালো লগা নাই বেয়াও লগা লা'

In [126]:
for i in range(100):
    print(f"Original value = {common_voice_test_transcription[i]['sentence'].lower()}")
    print(f"Decoded value' = {decoded_voice(common_voice_test[i]['input_values'])}")
    print('-------------------------')

Original value = আমাৰ ঘৰত তলা মৰা অৱস্থাত ওছৰৰে ঘৰত মানুহ নথকা সময়ত চুৰি হৈছিল।
Decoded value' = আমাৰ ঘৰত তলামৰা আবস্থাচৰাৰে ঘৰত মানুহনথকা হ'মাত চুৰি উছিল
Original value = মন ভালো লগা নাই, বেয়াও, নগা নাই।
Decoded value' = মোন ভালো লগা নাই বেয়াও লগা লা
Original value = ইয়াৰ পাচত সূৰ্য্যই যেতিয়া ৰোগীক কেনে দেখিলে সুধিলে ডাক্তৰে ক'ব লগাত পৰিল।
Decoded value' = ইয়াৰ পাচত সুজযে যেইতেয়া ৰুগিক কেনে দেখিলে সুধিলে ডাকাৰ ক'ব লগাত পৰিল
Original value = তেওঁ ৰামৰ এজন ভাল অনুগামী।
Decoded value' = তেওঁ ৰামৰ এজন ভাল কোনো গামি
Original value = ইউনেস্কোৰ দ্বাৰা স্বীকৃত প্ৰাকৃতিক বিশ্ব ঐতিহ্যবাহী স্থান মানস ৰাষ্ট্ৰীয় উদ্যান।
Decoded value' = ইমেশ্গ'ৰ যাৰামিপৃত প্ৰাককৃতিক বিশ্য্যইৱায়ী চান মানস ৰাষ্তৰীয় উদ্যা
Original value = এইদৰে বহুদিন গ'ল।
Decoded value' = ইগৰে বহুদিন গোল
Original value = যদিও এই উদ্যোগৰ ইতিহাস পুৰণি।
Decoded value' = যদিও এই উদ্দ্যুগৰ ইতিহাস পুৰলি
Original value = পুৰণিয়ে নতুনক কয় যে পুৰণিক এৰিলে
Decoded value' = পুৰণিয়ে নতুনক কয়  পুৰণিক এৰিলে
Original value = পুৱাতে মোক খাটাংকৈ 

Decoded value' = খাৰণ তেনে ৰচনা সেই বয়সত দমাৰ্দাৰাই ৰচিত হোৱাটো সম্ভৱ নাছিল
Original value = সি চাগে তোমাৰ কথা শুনা নাই আগতে।
Decoded value' = সি চাগে তোমাৰ কথা শুনা নাই আগতে
Original value = তেওঁৰ প্ৰধান লক্ষ্য আছিল পুতেকক অলপ সৰহীয়াকৈ খেতিৰ মাটি লৈ দিয়া।
Decoded value' = তেওঁৰ প্ৰধা লক্ষ আছিল পুতেকলপ সৰ িয়াকৈ খেতিৰ মাতিলৈ দিয়া
Original value = যে এই মুদ্ৰা পূৰ্ব ভাৰতৰ ব্ৰঙ্গৰ শাসনকৰ্তা চন্দ্ৰকেতুৰ।
Decoded value' = যেয়েই মিদ্ৰা পূৰ্বভাৰতৰ ব্ৰংগৰ সাসণনকৰে সন্ৰগেতো
Original value = পূৰ্ব্বকালত আমাৰ পূৰ্ব্বপুৰুষবিলাকে পিতৃক দেৱতুল্য সন্মান কৰিছিল।
Decoded value' = পূৰ্বগালত আমাৰ পূ্বপোশ বিলাকে পিত্ৃক দ়্াৱাথৈণ্য সন্মান কৰিছিঁ
Original value = সুত নলওঁ বুলিছে যদিও শেহত কি কৰে কোনে জানে!
Decoded value' = সূট্ননো বুলিছে যদিও সিহঁত কি কৰে কোনে জানে
Original value = ক'লে আৰু তোক কেতিয়াও খাবলৈ নিদিওঁ।
Decoded value' = ক'লোৰটোক কেতিযয়াও খাবলৈনিদিওঁঁ
Original value = ভূধৰ চিন্তাত নিমগ্ন হ'ল।
Decoded value' = ভু ধৰ চিন্তা নিবগ নগ'ল
Original value = এনেতে এদিন ৰাতি মােৰ বােৰ টোপনিৰ অৱস্থাত এটি আ

Decoded value' = টমৰ ফালে চাই ক'লে তম তোমাক মই একু দিব নোৱাৰোঁ
Original value = বাছা উলটি আহ।
Decoded value' = বাছা উলধিয়াহ
Original value = বটলত গৰম পানী ভৰাই লৈ আনিছোঁ।
Decoded value' = বটলত গৰম পানী ভৰাই লৈ আনিছোঁ
Original value = লেখা বুলি আগতে দেখাই আহিছে
Decoded value' = লেখা বুলি আগতে দেখাই আহিছে
Original value = কিন্তু বহুত দিনলৈকে কোনো চাকৰি যোগাৰ কৰিব নোৱাৰিলে।
Decoded value' = কিন্তু বহুত দিনলৈকে কোনো চা কৰি দৱাৰ কৰিবনোৱাৰিলে
Original value = তেওঁলোকে কথা পাতি থাকোঁতেই হৰৰ ঘৈণীয়েকে আৰু দুখনমান আঞ্জা নকৈ ৰান্ধিলে।
Decoded value' = তেওঁলোকে কথা পাতি ঠাকোতেই শৰৰ ঘৈণীয়েকে আৰু দুখনমান আন্জা মঁকৈ ৰানিলে
Original value = বহল অৰ্থত অসম ৰাজ্যৰ পৰা নিৰ্মাণ কৰা সকলো চলচ্চিত্ৰকে অসমৰ চলচ্চিত্ৰ বুলিব পাৰি।
Decoded value' = বহল অৰ্থত অসম ৰাজ্যাৰ পৰা নিৰ্মাণ কৰা সকলোচলচিতত্ৰকে অসমৰ্চলচিত্ৰ বুলিব পাৰি
Original value = দৰজাখন খুলিব নোৱাৰি।
Decoded value' = দৰ্জীখন খুলিব নোৱাৰি
Original value = কিন্তু ককাৰ খং তুলি চাবৰ সপক্ষে এনে ব্ৰহ্মাস্ত্ৰ আমাৰ হাতত আৰু আন এটা নাছিল।
Decoded value' = কিন্

Decoded value' = যি ব্যকিয়ে ফজৰৰ পূবে ৰুজাৰ কিয়ত নকৰে তাৰকো গোজা নাই
Original value = লক্ষ্যহীন জীৱন বঠা নকথা নাৱৰ দৰে।
Decoded value' = লৈ্ষসীন জীৱন বথানভৰগাঁ নাৱৰ দৰে
Original value = কিন্তু মই সদায়েই গভীৰ বিশ্বাসী আত্মীয়সকলৰদ্বাৰা আবৃত আছিলোঁ।
Decoded value' = কিন্তুমৰ সদাযয়েই গঢ়ীৰ বিষ্ছাশি  আত্মীযসকলৰ দাৰা আৰৃত্থ আছিলো
Original value = ঘৈণীয়েকে ভিতৰৰ পৰা সকলো শুনিছিল।
Decoded value' = ঘৈণীয়েকে ভিতৰৰ পৰা সকলুশুনিছিল
Original value = সমৰ্থকৰ সংখ্যাও বহুত বেছি।
Decoded value' = অমৰ্থকৰ সায়ীথাও বহুত বেচি
Original value = অসমীয়া সাধাৰণ গাঁৱলীয়া ছোৱালীৰনো চৰিত্ৰবল কেনে
Decoded value' = অসুমীয় সাধান গাঁৱইয়া ছোৱালীৰনী চূৰিত্ৰকলকে নেই
Original value = শংকৰদেৱে দুবাৰ তীৰ্থ ভ্ৰমণ নকৰা হ'লে তেওঁক হয়তো এজন বিদগ্ধ পণ্ডিত ৰূপত নাপালোঁহেঁতেন।
Decoded value' = সংকৰ দিৱি দুবাৰ তিন্্ষ বমাৰ নকৰা হ'লে তেওঁকৰটোৱেজন বিভক্ষ পদ্নি ৰূপত পাপালোহৈতেন
Original value = পিতাকক কিৰ্পিন বুলি হাঁহে সেই লাজতে পুতেকে বাগিচাৰ কামত আৰু হাত দিয়া নাই।
Decoded value' = পীতাকক  কিৰ্পিণ্বিলি হাহে সেই ৰাজতে আমুতেকে বাগি

Decoded value' = মানতোৱে ক'লে ভালকৈ শিক নিশি কিমি
Original value = কোনো প্ৰকাৰে শিয়ালটোৱ দোণটো ডিঙিৰ পৰা গুচাব নোৱাৰিলে।
Decoded value' = কোনো প্ৰকাৰে সিয়ালটোক দোনটো ডিঙিৰ পৰা গুচাব নোৱাৰিলে
Original value = পাণ্ডুত ৰেলৰ ডাঙৰ ষ্টেচন আৰু জাহাজঘাট।
Decoded value' = পান্ডুত ৰেলৰ ডাঙৰ ষ্টেচন আৰু জাহানখাত
Original value = ডালিমীৰ হৃদয়ত আকৌ দুটি ভাবে খেলিব ধৰিলে।
Decoded value' = ডালিমিৰ হিদয়ত আকৌ দুটি ভাবেশ খেলিব ধৰিলে
Original value = সংসাৰৰ আন কাম-কাজ বা চিন্তা-ভাৱনাৰ পৰা নিজকে আঁতৰাই ৰাখিছে।
Decoded value' = সংসাৰৰ আন কাম কাছ বাচচিন্তামোবুণাৰ পৰা নিজকে আঠৰাই ৰাখিছ
Original value = তেতিয়া মাক-জীয়েকে ভাত খালে আৰু শয্যাৰ আশ্ৰয় ল'লে।
Decoded value' = তেতিয়া মাকজীয়েকে ভাত খালে আৰু সজ্যাৰ আইশ্ৰই লেলে
Original value = আজি এয়া মহাৰাষ্ট্ৰৰ এক আমোদজনক ক্ৰিড়া।
Decoded value' = আজি এয়া মহাৰাষ্টৰ এক আমোধ চলক গ্ৰৃয়া
Original value = যেতিয়া এখন ঘৰৰ গৃহিণী হ'বা
Decoded value' = চিতিয়াএখন ঘৰৰ গৃহিণী হ'বা
Original value = তই এতিয়া ভৰ বয়সত মোৰ লগত যুঁজি মৰিবলৈ আহিনে?
Decoded value' = তই এতিয়া ভৰ বয়সত মোৰ

Decoded value' = ি সনজীৱত পিতীযা তুমিকেতিয় বেবহৰ কৰিব নোৱাৰিবা আৰু কুৰি লও সফল হ'ব নোৱাৰিব
Original value = এন্ধাৰ নিশা নাও এখন লৈ ঘাটত উপস্থিত হ'ল।
Decoded value' = এন্ঠাৰ নিশানন নাও এখন লৈ খাতত উপস্থিত হ'ল
Original value = বিয়াৰ বন্দবস্ত নোহোৱাত মদনে এইবাৰ কলিয়াবৰৰ ফালে চেষ্টা কৰিবলৈ মন মেলিলে।
Decoded value' = বিয়াৰ বৰ্দবস্টু নোহোৱাত মদনে এই বাৰ কলিয়া বৰৰ ফালে চেষ্টা কৰিবলৈ মন মেলিলে
Original value = বাপেকে কোনফালে মানুহটো পলাই গ'ল পুতেকক সুধিলে
Decoded value' = বাফেকে কোনফলে মানসটো পৰাই গ'ল পূঠেকক সুধহিলে
Original value = এই ছেটিং সলনি কৰাটো আপোনাৰ ছিষ্টেমৰ বাবে ক্ষতিকাৰক।
Decoded value' = ই চিটিং সলোনি লৱাটো আপোনাৰ ছিষ্টেমৰ বাবে খৰৰতি কাৰক
Original value = তাৰ চালৰ খেৰবোৰ উৱঁলি গৈছে দেখি উৰুখি বৰষুণত পানী পৰে।
Decoded value' = তাৰ চালৰ খেৰবোৰ উইি গৈছি দেখি উৰুখিবৰষুণত পানী পৰে
Original value = ইয়াক উত্তৰ-পূব ভাৰতৰ 'প্ৰৱেশদ্বাৰ' বুলিও কোৱা হয়।
Decoded value' = ইয়াত ত্ৰ'প উভাৰেও' প'বেএশ্ডাৰ গুলৈ কুৱা হৈ
Original value = য'তেই এনে অংশ পৰিল, তাতেই একোখন তীৰ্থস্থান গঢ়ি উঠিল।
Decoded 

Decoded value' = ডাঙৰ হৈ মাল প্লিয়া চলাম
Original value = শিৱসাগৰত আজি বতৰ কেনেকুৱা?
Decoded value' = সিৱ সগৰত আজিবটো কেনে কুৱা
Original value = মাছটোৱে আহি আনবোৰ মাছক বিলখনৰ সৌন্দৰ্যৰ কথা জনালে।
Decoded value' = মাষটুে আহি আনবোৰ মাছক বিলখনৰ কলৈজ্যৰ কথা জনালে
Original value = কৈছিল গংগাৰ পাৰত শ্যামলালক চিতাজুইত সিহঁতে নিজ হাতে তুলি দি আহিছিল।
Decoded value' = কৈছিল গোংগাৰ পাৰত ছিয়ামলালক চিটা যৈত সেঘত েনি ছাতে তুলি দি আহিছিল
Original value = লেগ্ৰীয়ে ক'লে- "এই টম, এই ফালে আহ।"
Decoded value' = লেগ্বীয়ে ক'লে ইক যান সিফেলে আহ
Original value = মাজিউটোৱে ইংৰাজী স্কুলৰ সপ্তম মানলৈকে পঢ়িছিল।
Decoded value' = মাজিওতোৱে ইংৰাজী ্কুলৰ শপ্কমমানলৈকে পহিছিল
Original value = সিও ওপৰৰ ডাঙৰ আখৰৰ শাৰীকেইটাৰ লেখীয়াকৈ তলৰ সৰু শাৰীকেইটাও সলসলীয়াকৈ মাতি দিলেহেঁতেন।
Decoded value' = সিও ওপৰৰ ডাঙৰ আসৰৰ খাৰীকেইটাৰ লিখিয়া অৈতলৰ সৰু খাৰীকেইটাও সলসলীযয়াকৈ মাতি দিলেহেঁতেন
Original value = তেওঁৰ লগত আগৰে চিনাকি আছে।
Decoded value' = তেওঁৰ লগত আগচে চিনাকি আছে
Original value = সুভদ্ৰা প্ৰথমতে মুছকঁছ যাওঁ যাওঁ হৈছিল আ

Decoded value' = সি দেখি ৰচ চকত সেইটোবৰকৈ উভহা যেন নেলাগিল
Original value = সেইমতে দুয়ো নিৰ্দিষ্ট স্থানৰ পৰা দৌৰ আৰম্ভ কৰিলে।
Decoded value' = সিমোত দুই উনিৰ্ডি্টিষ থানৰ পৰা দ'ৰ অনভকুইিলে
Original value = যদি ভালমতে খেতি হয় দুবছৰতে ধাৰ মৰি যাব।
Decoded value' = যুদি ভালমতে খেতি হয় দুশ বোছৰতে ধাৰ মৰি যাব
Original value = কিন্তু সিহঁতে তেতিয়াও নাজানিছিল সিহঁতকনো কি কৰিব।
Decoded value' = কিন্তু সিয়তে তেতিয় মাজানিছিল সিয়ঁতক নুকি কৰিব
Original value = তেতিয়া হ'লে পৰহিলৈ এইখিনি পাবহি।
Decoded value' = তিয়া হ'লে পুৰহিল সইখিনি পাবহি
Original value = আৰম্ভণিতে জিলাখনৰ মহকুমা আছিল দুটা- গোৱালপাৰা আৰু উত্তৰ শালমাৰা মহকুমা।
Decoded value' = আৰম্ভুনীতে জিলাখনৰ মহকুমা আছিল দুটা গোৱালপাৰাত আৰু উত্তৰ শালমা মহকুল
Original value = আমাৰ বাবে আছিল মাথোঁ সাধুকথা।
Decoded value' = আমাৰ বাবে আছিল পাথো হাত  কথা
Original value = যদি তোক এৰি দিব লাগে তেনেহ'লে মোক কি দিবি আগেয়েই দে।
Decoded value' = যদিটোক এইৰি দিব লাগে তেিনেহৈ মোখ কি দিবি আগেই দিে
Original value = দুখত সুখত সকলো সময়ত জানিবা কিতাপ তোমালোকৰ বন

Decoded value' = এনেকৈ বুঢাক সুৰুছোৱালীজনী দিয়াত বহুত পাপত্তি কুৰিছিল কিতুনা মানিলে
Original value = জুহালৰ ওচৰৰ বহল বেঞ্চ খনতে এখন বিছনা পাৰি দিয়া হ'ল।
Decoded value' = যোহালৰ ওচৰৰ বহল বেচজখনতে এখন বিছনা পাৰি দিয়া হ'ল
Original value = আজি বাৰ্থডে' কেক কাটিবানে?
Decoded value' = আজি বনু্তি কিক কাটিবানে
Original value = “কমাৰে যে কুঠাৰ নিদিয় মইনো দিম কিয়?”
Decoded value' = কমাৰেজে কুঠাৰ নিদিয় মলোু দিম কিয়
Original value = মই বুজিছোঁ, তাক চপনীয়া চাপিবলৈ ভিনীহিয়েকে হাক দিলে।
Decoded value' = মই বুজিছো তাক চপুনি চাপিবলৈ ভিনীয়ে কি ফাগ দিলে
Original value = এনে প্ৰশ্নৰ কথা কেনেও ভবা নাছিল।
Decoded value' = এনে প্ৰশ্ণেৰ কথা কেনেও খুৱাৰ আছিল
Original value = যন্ত্ৰণাত ছট্ফটাই এসময়ত বগলীটো মৰি থাকিল।
Decoded value' = জন্তনাথ চত ফতাই এসময় ন বপকলীটো মৰি থাকিল
Original value = এই সময়খিনিক ‘কৃত্ৰিম বুদ্ধিমত্তাৰ শীতকাল’ বুলি কোৱা হৈছিল।
Decoded value' = এই সমহিন্কেক্তিম পুৰ্ধিমতাৰ সিট্কাল বুলি কুৱা হৈছিল
Original value = স্কুলৰ শিক্ষকজনে সন্দেহ প্ৰকাশ কৰিছে, শিশুটিয়ে ভালদৰে দূৰৰ বস্তু বোধহয় দেখা

In [127]:
import csv

In [128]:
with open("decoded_voice.csv", "w", newline="") as f:
    writer = csv.writer(f)

    
    writer.writerow(["Original value", "Decoded value"])

    for i in range(len(common_voice_test_transcription)):
        original_value = common_voice_test_transcription[i]["sentence"].lower()
        decoded_value = decoded_voice(common_voice_test[i]["input_values"])

        writer.writerow([original_value, decoded_value])

        
f.close()

In [129]:
import pandas as pd

In [130]:
df = pd.read_csv('decoded_voice.csv')

In [131]:
df

,Original value,Decoded value
0,আমাৰ ঘৰত তলা মৰা অৱস্থাত ওছৰৰে ঘৰত মানুহ নথকা ...,আমাৰ ঘৰত তলামৰা আবস্থাত ওচৰেৰে ঘৰত মানুহনথকা হ...
1,"মন ভালো লগা নাই, বেয়াও, নগা নাই।",মোন ভালো লগা নাই বেয়াও লগা না
2,ইয়াৰ পাচত সূৰ্য্যই যেতিয়া ৰোগীক কেনে দেখিলে সু...,ইয়াৰ পাচত সুইজযচেতেয়া ৰুগি কেনে দেখিলে সোধাে ড...
3,তেওঁ ৰামৰ এজন ভাল অনুগামী।,তেওঁ আামৰ এজন ভাল খোনো গামি
4,ইউনেস্কোৰ দ্বাৰা স্বীকৃত প্ৰাকৃতিক বিশ্ব ঐতিহ্...,ইমেশগৰ যাৰাশিক্দয় প্ৰাকপৃতিক বিশ্য্যইৱা়ী চান...
...,...,...
546,সপ্তদশ শতাব্দীত ব্ৰিটিছসকলে এই অঞ্চলৰ অধিকাৰ অ...,সপ্তৰা সতাহিত পৃতিশ্সকুলে এই অঞ্চলৰ অধিকাৰ উৰ্...
547,ক্লাউড ষ্ট'ৰেজ বা স্থানীয় ষ্ট'ৰেজত ছেভ কৰক।,ক্লাটটৰিজ পেষ্থােষ্টৰিজথ্ছি ফৰক
548,আমেৰিকাৰ গৃহযুদ্ধৰ সময়ত বোম্বাই বিশ্বৰ প্ৰধান...,আমেৰিকাৰ গ্ৰৃহ্যুত্ৰ সময়ত বোংৰ পিশ্বৰ প্ৰৰান ক...
549,নাৱেদি আহোঁতে বাটত দেউতাই আঙুলিয়াই আমাক দেখুৱ...,নাৱেলী আহোঁতে বাটত দেউতাই আমুলীয়া আমাং দিশোৱা ...
